# Headline

In [79]:
from notebook.services.config import ConfigManager
cm = ConfigManager().update('notebook', {'output_length': 100})

In [80]:
# importing the main packages
import numpy as np
import pandas as pd
import pandas_datareader
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
import os
import datetime


## Raw Data

In [81]:
# from datetime import datetime

In [82]:
from os import listdir
from os.path import isfile, join
filenames_eia = [f for f in listdir("raw_data/eia_data") if isfile(join("raw_data/eia_data", f))]
filenames_eia

['U.S._Field_Production_of_Crude_Oil_Monthly.csv',
 'U.S._Ending_Stocks_of_Total_Gasoline_Monthly.csv',
 'U.S._Exports_of_Crude_Oil_Monthly.csv',
 'U.S._Ending_Stocks_of_Crude_Oil_Monthly.csv',
 'Regular_Gasoline_Retail_Prices_Monthly.csv',
 'U.S._Imports_of_Crude_Oil_Monthly.csv',
 'U.S._Product_Supplied_of_Finished_Motor_Gasoline_Monthly.csv']

In [83]:
filenames = [f for f in listdir("raw_data") if isfile(join("raw_data", f))]
filenames

['XOM.csv',
 'CPIAUCSL.csv',
 'CVX.csv',
 'SP500.csv',
 'MacroTrends_Data_Download_CVX.csv',
 'MacroTrends_Data_Download_XOM.csv']

In [84]:
gas_prices = pd.read_csv('raw_data/eia_data/' + 'Regular_Gasoline_Retail_Prices_Monthly.csv', skiprows=4)
gas_prices["Date"]= pd.to_datetime(gas_prices["Month"])
gas_prices.set_index('Date', inplace=True)
gas_prices.drop(columns=["Month"], inplace=True)
gas_prices.sort_index(inplace=True)
first = gas_prices.index.min()
before_first = first - datetime.timedelta(days=31) 
last = gas_prices.index[-21]
# offsetting prices by one month to make comparison logic/concatenating easier
# we want to know if the price for the next month is higher/lower based
# on info from this month
gas_prices.index = gas_prices.index +pd.DateOffset(months=1)


In [85]:
gas_prices = gas_prices.iloc[gas_prices.index.get_loc(gas_prices.index.min()):gas_prices.index.get_loc(last)+1]

In [86]:
production = pd.read_csv('raw_data/eia_data/' + 'U.S._Field_Production_of_Crude_Oil_Monthly.csv', skiprows=4)
production["Date"]= pd.to_datetime(production["Month"])
production.set_index('Date', inplace=True)
production["Crude Production"] = production["Series ID: PET.MCRFPUS1.M Thousand Barrels"]
production.drop(columns=["Month","Series ID: PET.MCRFPUS1.M Thousand Barrels"], inplace=True)

In [87]:
stocks_gas = pd.read_csv('raw_data/eia_data/' + 'U.S._Ending_Stocks_of_Total_Gasoline_Monthly.csv', skiprows=4)
stocks_gas["Date"]= pd.to_datetime(stocks_gas["Month"])
stocks_gas.set_index('Date', inplace=True)
stocks_gas["Stocks Gas"] = stocks_gas["Series ID: PET.MGTSTUS1.M Thousand Barrels"]
stocks_gas.drop(columns=["Month","Series ID: PET.MGTSTUS1.M Thousand Barrels"], inplace=True)

In [88]:
gasoline_supplied = pd.read_csv('raw_data/eia_data/' + 'U.S._Product_Supplied_of_Finished_Motor_Gasoline_Monthly.csv', skiprows=4)
gasoline_supplied["Date"]= pd.to_datetime(gasoline_supplied["Month"])
gasoline_supplied.set_index('Date', inplace=True)
gasoline_supplied["Gasoline Supplied"] = gasoline_supplied["Series ID: PET.MGFUPUS1.M Thousand Barrels"]
gasoline_supplied.drop(columns=["Month","Series ID: PET.MGFUPUS1.M Thousand Barrels"], inplace=True)


In [89]:
stocks_crude = pd.read_csv('raw_data/eia_data/' + 'U.S._Ending_Stocks_of_Crude_Oil_Monthly.csv', skiprows=4)
stocks_crude["Date"]= pd.to_datetime(stocks_crude["Month"])
stocks_crude.set_index('Date', inplace=True)
stocks_crude["Stocks Crude"] = stocks_crude["Series ID: PET.MCRSTUS1.M Thousand Barrels"]
stocks_crude.drop(columns=["Month","Series ID: PET.MCRSTUS1.M Thousand Barrels"], inplace=True)


In [90]:
exports = pd.read_csv('raw_data/eia_data/' + 'U.S._Exports_of_Crude_Oil_Monthly.csv', skiprows=4)
exports["Date"]= pd.to_datetime(exports["Month"])
exports.set_index('Date', inplace=True)
exports["Exports"] = exports["Series ID: PET.MCREXUS1.M Thousand Barrels"]
exports.drop(columns=["Month","Series ID: PET.MCREXUS1.M Thousand Barrels"], inplace=True)


In [91]:
imports = pd.read_csv('raw_data/eia_data/' + 'U.S._Imports_of_Crude_Oil_Monthly.csv', skiprows=4)
imports["Date"]= pd.to_datetime(imports["Month"])
imports.set_index('Date', inplace=True)
imports["Imports"] = imports["Series ID: PET.MCRIMUS1.M Thousand Barrels"]
imports.drop(columns=["Month","Series ID: PET.MCRIMUS1.M Thousand Barrels"], inplace=True)

In [92]:
cpi = pd.read_csv('raw_data/CPIAUCSL.csv')
cpi["Date"]= pd.to_datetime(cpi["DATE"])
cpi.set_index('Date', inplace=True)
cpi.drop(columns=["DATE"], inplace=True)


In [93]:
exxon = pd.read_csv('raw_data/MacroTrends_Data_Download_XOM.csv', skiprows=14)
exxon["Date"]= pd.to_datetime(exxon["date"])
exxon.set_index('Date', inplace=True)
# to avoid data loss due to the first being a non-trading day, we use monthly averages
close = exxon["close"].groupby(pd.Grouper(freq='M')).mean()
volume = exxon["volume"].groupby(pd.Grouper(freq='M')).mean()
exxon = pd.concat([close, volume], axis=1).reset_index()
exxon.set_index('Date', inplace=True)
exxon.index = [pd.datetime(x.year, x.month, 1) for x in exxon.index.tolist()]
exxon.rename(columns={"close": "Exxon", "volume": "Exxon Volume"}, inplace=True)


/tmp/ipykernel_7094/3883403308.py:9: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.



In [94]:
chevron = pd.read_csv('raw_data/MacroTrends_Data_Download_CVX.csv', skiprows=14)
chevron["Date"]= pd.to_datetime(chevron["date"])
chevron.set_index('Date', inplace=True)
# to avoid data loss due to the first being a non-trading day, we use monthly averages
close = chevron["close"].groupby(pd.Grouper(freq='M')).mean()
volume = chevron["volume"].groupby(pd.Grouper(freq='M')).mean()
chevron = pd.concat([close, volume], axis=1).reset_index()
chevron.set_index('Date', inplace=True)
chevron.index = [pd.datetime(x.year, x.month, 1) for x in chevron.index.tolist()]
chevron.rename(columns={"close": "Chevron", "volume": "Chevron Volume"}, inplace=True)

/tmp/ipykernel_7094/46727539.py:9: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.



In [95]:
sp500 = pd.read_csv('raw_data/SP500.csv')
sp500["Date"]= pd.to_datetime(sp500["Date"])
sp500.set_index('Date', inplace=True)
# to avoid data loss due to the first being a non-trading day, we use monthly averages 
sp500["SP500"] = sp500["Value"]
sp500.drop(sp500.columns.difference(['SP500']), 1, inplace=True)
sp500

/tmp/ipykernel_7094/77266523.py:6: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



,SP500
Date,
2022-07-01,3825.33
2022-06-30,3785.38
2022-06-01,3789.99
2022-05-31,4132.15
2022-05-01,4040.36
...,...
1871-05-01,4.86
1871-04-01,4.74
1871-03-01,4.61


In [96]:
def cut(df):
    first = before_first
    df.sort_index(inplace=True)
    if first not in df.index:
        first = df.index.min()
    df = df.iloc[df.index.get_loc(first):df.index.get_loc(last)+1]
    return df

In [97]:
chevron = cut(chevron)
cpi = cut(cpi)
production = cut(production)
exports = cut(exports)
exxon = cut(exxon)
gas_prices = cut(gas_prices)
imports = cut(imports)
sp500 = cut(sp500)
stocks_crude = cut(stocks_crude)
stocks_gas = cut(stocks_gas)
gasoline_supplied = cut(gasoline_supplied)


In [98]:
df_list = [cpi, gas_prices, production, gasoline_supplied, stocks_gas, exports, imports, sp500, chevron, exxon]
df = pd.concat(df_list, axis=1)
df = df.iloc[df.index.is_month_start]
df.dropna(inplace=True)
df

,CPIAUCSL,Real Values,Nominal Values,Crude Production,Gasoline Supplied,Stocks Gas,Exports,Imports,SP500,Chevron,Chevron Volume,Exxon,Exxon Volume
1976-02-01,55.900,3.15,0.61,238705.0,181641.0,248901.0,0.0,122030.0,100.60,2.077605,9.581053e+05,0.836537,3.190737e+06
1976-03-01,56.000,3.12,0.60,255198.0,213605.0,239097.0,45.0,146879.0,101.10,2.079065,1.009878e+06,0.842430,2.809739e+06
1976-04-01,56.100,3.08,0.59,242310.0,214782.0,224008.0,0.0,143687.0,101.90,2.297600,1.082324e+06,0.876167,3.239314e+06
1976-05-01,56.400,3.06,0.59,251860.0,212442.0,225079.0,1.0,144748.0,101.20,2.419110,8.908000e+05,0.923040,3.759840e+06
1976-06-01,56.700,3.09,0.60,242833.0,224458.0,225408.0,0.0,168833.0,101.80,2.468064,7.161455e+05,0.966559,2.940509e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-01,280.126,3.54,3.39,359709.0,277432.0,232237.0,107023.0,199068.0,4674.77,114.196205,1.004029e+07,59.990477,1.965705e+07
2022-01-01,281.933,3.43,3.31,352449.0,247450.0,251753.0,103760.0,197873.0,4573.82,125.147300,1.502323e+07,69.839955,3.034585e+07
2022-02-01,284.182,3.41,3.31,316851.0,240744.0,250431.0,92654.0,172325.0,4435.98,134.289995,1.486751e+07,77.811553,3.306380e+07
2022-03-01,287.708,3.59,3.52,362329.0,274538.0,238472.0,102883.0,198883.0,4391.27,161.821826,2.292033e+07,81.507139,3.707830e+07


In [99]:
df["Crude Production to Real Price"]=df["Crude Production"]/df["Real Values"]
df["Crude Production to Exports"]=df["Crude Production"]/df["Exports"]
df["Crude Production to Imports"]=df["Crude Production"]/df["Imports"]
df["Crude Production to Stocks"]=df["Crude Production"]/df["Stocks Gas"]


df["Gasoline Supplied to Crude Production"]=df["Gasoline Supplied"]/df["Crude Production"]
df["Gasoline Supplied to Crude Exports"]=df["Gasoline Supplied"]/df["Exports"]
df["Gasoline Supplied to Crude Imports"]=df["Gasoline Supplied"]/df["Imports"]
df["Gasoline Supplied to Stocks Gas"]=df["Gasoline Supplied"]/df["Stocks Gas"]

df["Real Price to Exxon"]=df["Real Values"]/df["Exxon"]
df["Real to Chevron"]=df["Real Values"]/df["Chevron"]
df["Real Price to SP 500"]=df["Real Values"]/df["SP500"]
df["Real Price to CPI"]=df["Real Values"]/df["CPIAUCSL"]

df["Movement"] = np.where((df["Real Values"]-df["Real Values"].shift())>0, 1, 0)

# replacing and dropping infinite values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

df

,CPIAUCSL,Real Values,Nominal Values,Crude Production,Gasoline Supplied,Stocks Gas,Exports,Imports,SP500,Chevron,Chevron Volume,Exxon,Exxon Volume,Movement
1976-02-01,55.900,3.15,0.61,238705.0,181641.0,248901.0,0.0,122030.0,100.60,2.077605,9.581053e+05,0.836537,3.190737e+06,0
1976-03-01,56.000,3.12,0.60,255198.0,213605.0,239097.0,45.0,146879.0,101.10,2.079065,1.009878e+06,0.842430,2.809739e+06,0
1976-04-01,56.100,3.08,0.59,242310.0,214782.0,224008.0,0.0,143687.0,101.90,2.297600,1.082324e+06,0.876167,3.239314e+06,0
1976-05-01,56.400,3.06,0.59,251860.0,212442.0,225079.0,1.0,144748.0,101.20,2.419110,8.908000e+05,0.923040,3.759840e+06,0
1976-06-01,56.700,3.09,0.60,242833.0,224458.0,225408.0,0.0,168833.0,101.80,2.468064,7.161455e+05,0.966559,2.940509e+06,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-01,280.126,3.54,3.39,359709.0,277432.0,232237.0,107023.0,199068.0,4674.77,114.196205,1.004029e+07,59.990477,1.965705e+07,1
2022-01-01,281.933,3.43,3.31,352449.0,247450.0,251753.0,103760.0,197873.0,4573.82,125.147300,1.502323e+07,69.839955,3.034585e+07,0
2022-02-01,284.182,3.41,3.31,316851.0,240744.0,250431.0,92654.0,172325.0,4435.98,134.289995,1.486751e+07,77.811553,3.306380e+07,0
2022-03-01,287.708,3.59,3.52,362329.0,274538.0,238472.0,102883.0,198883.0,4391.27,161.821826,2.292033e+07,81.507139,3.707830e+07,1


In [100]:
import plotly.express as px
fig = px.line(df, title='')
fig.show()

In [101]:
df_change = (df.pct_change())
fig = px.line(df_change, title='Percent change of variables')
fig.show()

In [102]:
fig = px.line(df_change.drop(columns=["Exports", "Imports", "Chevron", "Exxon"]), title='Log of percent change of variables')
fig.show()

In [103]:
fig = px.line(df[["Nominal Values", "Real Values"]], title='Percent change of variables')
fig.show()

In [104]:

df

,CPIAUCSL,Real Values,Nominal Values,Crude Production,Gasoline Supplied,Stocks Gas,Exports,Imports,SP500,Chevron,Chevron Volume,Exxon,Exxon Volume,Movement
1976-02-01,55.900,3.15,0.61,238705.0,181641.0,248901.0,0.0,122030.0,100.60,2.077605,9.581053e+05,0.836537,3.190737e+06,0
1976-03-01,56.000,3.12,0.60,255198.0,213605.0,239097.0,45.0,146879.0,101.10,2.079065,1.009878e+06,0.842430,2.809739e+06,0
1976-04-01,56.100,3.08,0.59,242310.0,214782.0,224008.0,0.0,143687.0,101.90,2.297600,1.082324e+06,0.876167,3.239314e+06,0
1976-05-01,56.400,3.06,0.59,251860.0,212442.0,225079.0,1.0,144748.0,101.20,2.419110,8.908000e+05,0.923040,3.759840e+06,0
1976-06-01,56.700,3.09,0.60,242833.0,224458.0,225408.0,0.0,168833.0,101.80,2.468064,7.161455e+05,0.966559,2.940509e+06,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-01,280.126,3.54,3.39,359709.0,277432.0,232237.0,107023.0,199068.0,4674.77,114.196205,1.004029e+07,59.990477,1.965705e+07,1
2022-01-01,281.933,3.43,3.31,352449.0,247450.0,251753.0,103760.0,197873.0,4573.82,125.147300,1.502323e+07,69.839955,3.034585e+07,0
2022-02-01,284.182,3.41,3.31,316851.0,240744.0,250431.0,92654.0,172325.0,4435.98,134.289995,1.486751e+07,77.811553,3.306380e+07,0
2022-03-01,287.708,3.59,3.52,362329.0,274538.0,238472.0,102883.0,198883.0,4391.27,161.821826,2.292033e+07,81.507139,3.707830e+07,1


## Data clean-up

In [105]:
def data_clean(start_date, end_date, columns):
    return df

In [106]:
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn import linear_model, tree, ensemble
from sklearn.ensemble import RandomForestClassifier

In [107]:
y = df["Movement"]
X = df.drop(columns=("Movement"))

In [108]:
rf_Model = RandomForestClassifier()

In [109]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=101)

In [110]:
print(f'X_train : {X_train.shape}')
print(f'y_train : {y_train.shape}')
print(f'X_test : {X_test.shape}')
print(f'y_test : {y_test.shape}')

X_train : (499, 13)
y_train : (499,)
X_test : (56, 13)
y_test : (56,)


In [111]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [2,4]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [112]:
# Create the param grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid)

{'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72, 80], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [113]:
rf_Model = RandomForestClassifier()

In [114]:
from sklearn.model_selection import GridSearchCV
rf_Grid = GridSearchCV(estimator = rf_Model, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)

In [115]:
rf_Grid.fit(X_train, y_train)

Fitting 3 folds for each of 320 candidates, totalling 960 fits
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=17; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=17; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=17; total time=   0.0s
[CV] END bootstrap=True, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   0.0s

GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=4,
             param_grid={'bootstrap': [True, False], 'max_depth': [2, 4],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72,
                                          80]},
             verbose=2)

In [116]:
rf_Grid.best_params_

{'bootstrap': True,
 'max_depth': 2,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 64}

In [117]:
print (f'Train Accuracy - : {rf_Grid.score(X_train,y_train):.3f}')
print (f'Test Accuracy - : {rf_Grid.score(X_test,y_test):.3f}')

Train Accuracy - : 0.655
Test Accuracy - : 0.518


## Machine Learning

### Random Forest

### Perceptron


## Results